### download the dataset

In [1]:
!pip install requests
!pip install certifi



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/8a/6a/19e9fe04fca059ccf770861c7d5721ab4c2aebc539889e97c7977528a53b/pip-24.0-py3-none-any.whl.metadata
  Using cached pip-24.0-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.0-py3-none-any.whl (2.1 MB)


ERROR: To modify pip, please run the following command:
C:\Users\riad\OneDrive\Desktop\py_env\pythonProject2\.venv\Scripts\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# download dataset

In [3]:
import requests


def download_ds(url, local_filename):
    try:
        response = requests.get(url, stream=True, verify=False)
        if response.status_code == 200:
            with open(local_filename, 'wb') as file:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        file.write(chunk)
            print(f"Downloaded file as {local_filename}")
        else:
            print(f"Failed to download file. Status code: {response.status_code}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [4]:
download_ds("https://datarepo.eng.ucsd.edu/mcauley_group/data/amazon_2023/raw/review_categories/All_Beauty.jsonl.gz",
            "All_Beauty.jsonl.gz")

C:\Users\riad\OneDrive\Desktop\py_env\pythonProject2\.venv\Lib\site-packages\urllib3\connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datarepo.eng.ucsd.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Downloaded file as All_Beauty.jsonl.gz


In [5]:
# prompt: unzip All_Beauty.jsonl.gz

!gunzip All_Beauty.jsonl.gz


'gunzip' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/22/a5/a0b255295406ed54269814bc93723cfd1a0da63fb9aaf99e1364f07923e5/pandas-2.2.2-cp312-cp312-win_amd64.whl.metadata
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.5 MB 640.0 kB/s eta 0:00:18
   ---------------------------------------- 0.1/11.5 MB 777.7 kB/s eta 0:00:15
    --------------------------------------- 0.2/11.5 MB 1.3 MB/s eta 0:00:09
    --------------------------------------- 0.2/11.5 MB 1.3 MB/s eta 0:00


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
import json


def filter_jsonl_file(input_file, output_file, max_lines=5000):
    with open(input_file, 'r') as f_in, open(output_file, 'w') as f_out:
        for i, line in enumerate(f_in):
            if i >= max_lines:
                break
            try:
                data = json.loads(line)
                if all(key in data for key in ('text', 'rating', 'title')):
                    print(line)
                    f_out.write(line)
            except json.JSONDecodeError:
                continue


# Usage
filter_jsonl_file('All_Beauty.jsonl', 'Filtered_All_Beauty.jsonl')

In [7]:
import pandas as pd
import json


def json_to_dataframe(json_data):
    data_list = []
    for line in json_data.split("\n"):
        if line.strip() != "":
            try:
                data = json.loads(line)
                data_list.append(
                    {"rating": data["rating"], "title": data["title"], "text": data["text"]}
                )
            except json.JSONDecodeError as e:
                print(f"An error occurred: {e}")
                print(f"An error occurred: {e.__traceback__}")
    df = pd.DataFrame(data_list)
    return df

In [8]:
# filter_jsonl_file('All_Beauty.jsonl', 'Filtered_All_Beauty.jsonl')
import pandas as pd
import json


def json_to_dataframe_rating_title(json_data):
    data_list = []
    for line in json_data.split("\n"):
        if line.strip() != "":
            try:
                data = json.loads(line)
                data_list.append(
                    {"rating": data["rating"], "title": data["title"]}
                )
            except json.JSONDecodeError as e:
                print(f"An error occurred: {e}")
                print(f"An error occurred: {e.__traceback__}")
    df = pd.DataFrame(data_list)
    return df

In [114]:
json_data = open("All_Beauty.jsonl", "r").read()
df = json_to_dataframe(json_data)

In [9]:
json_data = open("All_Beauty.jsonl", "r").read()
df = json_to_dataframe_rating_title(json_data)

In [10]:
print(df.head())

   rating                                      title
0     5.0  Such a lovely scent but not overpowering.
1     4.0     Works great but smells a little weird.
2     5.0                                       Yes!
3     1.0                          Synthetic feeling
4     5.0                                         A+


In [11]:
!pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


# training using nn

In [12]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
# import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from joblib import dump

print(df.head())

# Combine 'title' and 'text' columns
df['title_text'] = df['title']
df = df.head(50000)
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['title_text'], df['rating'], test_size=0.2, random_state=42)
print("starting training")
# Create a pipeline that first transforms the 'title_text' into feature vectors then trains the classifier
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MLPClassifier(verbose=True)),  # Set warm_start to True
])
# for i in range(100):  # Replace 100 with the number of iterations you want
text_clf.fit(X_train, y_train)
# dump(text_clf, f'model_checkpoint_{i}.joblib')
# save the model
try:
    dump(text_clf, 'model.joblib')
except Exception as e:
    print(f"An error occurred in saving the model: {e}")

# Predict the test set results
y_pred = text_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy: {accuracy}")

   rating                                      title
0     5.0  Such a lovely scent but not overpowering.
1     4.0     Works great but smells a little weird.
2     5.0                                       Yes!
3     1.0                          Synthetic feeling
4     5.0                                         A+
starting training
Iteration 1, loss = 1.08050602
Iteration 2, loss = 0.76024054
Iteration 3, loss = 0.67934319
Iteration 4, loss = 0.63167882
Iteration 5, loss = 0.59577530
Iteration 6, loss = 0.56789871
Iteration 7, loss = 0.54525418
Iteration 8, loss = 0.52622481
Iteration 9, loss = 0.51064498
Iteration 10, loss = 0.49661005
Iteration 11, loss = 0.48461882
Iteration 12, loss = 0.47334021
Iteration 13, loss = 0.46286208
Iteration 14, loss = 0.45376132
Iteration 15, loss = 0.44418076
Iteration 16, loss = 0.43550938
Iteration 17, loss = 0.42689391
Iteration 18, loss = 0.41882546
Iteration 19, loss = 0.41084278
Iteration 20, loss = 0.40342182
Iteration 21, loss = 0.39566023
I

C:\Users\riad\OneDrive\Desktop\py_env\pythonProject2\.venv\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model accuracy: 0.6835


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from joblib import dump

# Combine 'title' and 'text' columns
df['title_text'] = df['title']
df = df.head(50000)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['title_text'], df['rating'], test_size=0.2, random_state=42)

# Create a pipeline that first transforms the 'title_text' into feature vectors then trains the classifier
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB()),  # Use MultinomialNB instead of MLPClassifier
])

# Train the classifier
text_clf.fit(X_train, y_train)

# Save the model
try:
    dump(text_clf, 'modelNB.joblib')
except Exception as e:
    print(f"An error occurred in saving the model: {e}")

# Predict the test set results
y_pred = text_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy: {accuracy}")

Model accuracy: 0.6903


**not accurate**

In [16]:
text_clf.predict(["i hate this"]) # not accurate

array([5.])

# using decision tree

In [17]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from joblib import dump

# Combine 'title' and 'text' columns
df['title_text'] = df['title']
df = df.head(50000)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['title_text'], df['rating'], test_size=0.2, random_state=42)

# Create a pipeline that first transforms the 'title_text' into feature vectors then trains the classifier
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier()),  # Use DecisionTreeClassifier instead of MultinomialNB
])

# Train the classifier
text_clf.fit(X_train, y_train)

# Save the model
try:
    dump(text_clf, 'modelDT.joblib')
except Exception as e:
    print(f"An error occurred in saving the model: {e}")

# Predict the test set results
y_pred = text_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy: {accuracy}")

Model accuracy: 0.6733


In [27]:

text_clf.predict(["not expensive", "so expensive"]) # not accurate but much better than naive bayes


array([1., 5.])

# using knn

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from joblib import dump

# Combine 'title' and 'text' columns
df['title_text'] = df['title']
df = df.head(50000)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df['title_text'], df['rating'], test_size=0.2, random_state=42)

# Create a pipeline that first transforms the 'title_text' into feature vectors then trains the classifier
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', KNeighborsClassifier()),  # Use KNeighborsClassifier instead of DecisionTreeClassifier
])

# Train the classifier
text_clf.fit(X_train, y_train)

# Save the model
try:
    dump(text_clf, 'modelKNN.joblib')
except Exception as e:
    print(f"An error occurred in saving the model: {e}")

# Predict the test set results
y_pred = text_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy: {accuracy}")

Model accuracy: 0.6762


In [36]:
text_clf.predict([
    "very cheap",
                 "very expensive",
    "not expensive",
    "love it",
    "hate it",
    "so good",
    "so bad",
    "so so",
    "not good",
    "not bad",
                  ]) # not accurate but much better than naive bayes

array([1., 1., 1., 5., 1., 2., 1., 3., 1., 4.])

# scaling the rating will not work properly

In [111]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPClassifier
from joblib import dump

# Combine 'title' and 'text' columns
df['title_text'] = df['title'] + " " + df['text']
df = df.head(200000)
print(df.head())
# Split the data into training and test sets
import random

# Scale ratings from 1-5 to 1-10
df['rating_scaled'] = df['rating'] * 2

# Generate a random second digit for each rating
df['second_digit'] = [random.randint(0, 9) for _ in range(len(df))]

# Combine the first and second digits to get the new rating
df['rating_final'] = df['rating_scaled'] * 10 + df['second_digit']

# Now use 'rating_final' for training
X_train, X_test, y_train, y_test = train_test_split(df['title_text'], df[' '], test_size=0.2, random_state=42)
# Create a pipeline that first transforms the 'title_text' into feature vectors then trains the classifier
text_clf = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MLPClassifier(verbose=True)),
])

# Train the classifier
text_clf.fit(X_train, y_train)
# save the model
try:
    dump(text_clf, 'model2_scaled.joblib')
except Exception as e:
    print(f"An error occurred in saving the model: {e}")
# Predict the test set results
y_pred = text_clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

print(f"Model accuracy: {accuracy}")

   rating                                      title  \
0     5.0  Such a lovely scent but not overpowering.   
1     4.0     Works great but smells a little weird.   
2     5.0                                       Yes!   
3     1.0                          Synthetic feeling   
4     5.0                                         A+   

                                                text  \
0  This spray is really nice. It smells really go...   
1  This product does what I need it to do, I just...   
2                          Smells good, feels great!   
3                                     Felt synthetic   
4                                            Love it   

                                          title_text  rating_scaled  \
0  Such a lovely scent but not overpowering. This...           10.0   
1  Works great but smells a little weird. This pr...            8.0   
2                     Yes! Smells good, feels great!           10.0   
3                   Synthetic feeling Felt

/home/saeed/Desktop/py_env/GA-NLP_rnn/.venv/lib/python3.12/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Model accuracy: 0.0664


In [8]:
!pip install joblib

In [1]:
import joblib

text_clf = joblib.load('model.joblib')

In [3]:

text_clf.predict(["makes me so good"])

array([5.])

In [67]:
import json
def save_dictionary_from_data_frame_to_json(df,column_name, filename):
    """
    this function takes a dataframe , and in it's column_name there should be a text and it will iterare 
    :param df: 
    :param filename: 
    :return: 
    """
    data_dict = dict()
    c = 0
    for sentence in df[column_name]:
        for word in sentence.split(" "):
            if word not in data_dict.keys():
                c+=1
            data_dict[word] = c
    with open(filename, 'w') as f:
        json.dump(data_dict, f)

In [78]:
import re

def save_dictionary_from_data_frame_to_txt(df, column_name, filename):
    """
    this function takes a dataframe, and in its column_name there should be text, and it will iterate
    :param df: DataFrame containing text data
    :param column_name: Name of the column in the DataFrame containing text
    :param filename: Name of the file to save the dictionary
    """
    data_dict = set()
    for sentence in df[column_name]:
        words = re.findall(r'\b\w+\b', sentence)  # Using regex to extract words
        data_dict.update(words)
    with open(filename, 'w') as f:
        for word in data_dict:
            f.write(f"{word}\n")


In [99]:
import re
def save_dictionary_from_data_frame_to_txt_most_n_times(df, column_name, filename, n):
    words_dict = dict()
    words_arr = []
    for sentence in df[column_name]:
        words = re.findall(r'\b\w+\b', sentence)  # Using regex to extract words
        words_arr.append(words)
    for sentence_p in words_arr:
        for w in sentence_p:
            if w in words_dict.keys():            
                words_dict[w] = words_dict[w] + 1 
            else:
                words_dict[w] = 1
    final_arr = []
    for key in words_dict.keys():
        if words_dict[key] >= n:
            final_arr.append(key)
    with open(filename, 'w') as f:
        for w in final_arr:
            f.write(f"{w}\n")

In [57]:
df["title_text"]

0        Such a lovely scent but not overpowering. This...
1        Works great but smells a little weird. This pr...
2                           Yes! Smells good, feels great!
3                         Synthetic feeling Felt synthetic
4                                               A+ Love it
                               ...                        
49995    These were too heavy for me. I found these a l...
49996    Fun to use Love playing around with different ...
49997    More cosmetics, less cake flavoring. Not what ...
49998    Beautiful, creamy, silky, stain for lips and c...
49999    Maybe better for kids’ hair. The scrunchies ar...
Name: title_text, Length: 50000, dtype: object

In [68]:
save_dictionary_from_data_frame_to_json(df, "title_text", "dictionary.json")

In [79]:
save_dictionary_from_data_frame_to_txt(df, "title_text", "dictionary.txt")

In [103]:
save_dictionary_from_data_frame_to_txt_most_n_times(df, "title_text", "new_dictionary.txt", 500)

In [81]:
import random

def get_random_words(filename, n):
    with open(filename, 'r') as f:
        words = f.read().splitlines()
    return random.sample(words, n)

# Usage
reduced_dict = get_random_words('dictionary.txt', 10000)
print(reduced_dict[:50])

['experience', 'Granbaby', 'employees', 'Stroping', 'Yaaaasssss', 'gunpowder', 'crossdressing', 'después', 'Mists', '108w', 'Alsome', 'blokes', 'defining', 'frizzing', 'patching', 'gifts', 'CANNOT', 'Carols', 'Flashlights', 'microfibers', 'BANDS', 'explore', 'festival', 'sassoon', 'Estheticians', 'scraps', 'Some', 'Sufficient', 'professionalism', 'sparingly', 'confidently', 'Wearing', 'eliminate', 'Ema', 'OFFICE', 'seconds', 'uneasy', 'WENT', '520', 'Theyre', 'oloring', 'BOX', 'bug', 'B01FE1VHPQ', '54d75c107f2c222477f776b2c0f01245', 'many', 'copay', 'fog', 'sis', 'quot']


In [104]:
def get_words(filename):
    with open(filename, 'r') as f:
        words = f.read().splitlines()
    return words

In [105]:
final_dictionary = get_words("new_dictionary.txt")

In [107]:
import numpy
import random
from deap import base, creator, tools, algorithms

# Create the fitness class, which will be the base of the individual
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Create the individual class based on list with fitness attribute
creator.create("Individual", list, fitness=creator.FitnessMax)
# Define the fitness function
def fitness(individual):
    sentence = " ".join(individual)
    score = text_clf.predict([sentence])[0]
    return score,

# Define the mutation function
def mutate(individual):
    index = random.randrange(len(individual))
    individual[index] = random.choice(reduced_dict)
    return individual,

# Create the toolbox
toolbox = base.Toolbox()
toolbox.register("individual", tools.initRepeat, creator.Individual, lambda: random.choice(final_dictionary), n=20)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", fitness)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

# Initialize a population
pop = toolbox.population(n=300)

# Define the hall of fame
hof = tools.HallOfFame(1)

# Perform the genetic algorithm
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", numpy.mean)
stats.register("min", numpy.min)
stats.register("max", numpy.max)

pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, stats=stats, halloffame=hof, verbose=True)
sentence = " ".join(hof[0])
# Print the best sentence found
print(sentence)
text_clf.predict([sentence])

/home/saeed/Desktop/py_env/GA-NLP_rnn/.venv/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/home/saeed/Desktop/py_env/GA-NLP_rnn/.venv/lib/python3.12/site-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals	avg    	min	max
0  	300   	4.23333	1  	5  
1  	184   	4.81   	1  	5  
2  	187   	4.88   	1  	5  
3  	177   	4.90333	1  	5  
4  	166   	4.93   	1  	5  
5  	176   	4.92333	1  	5  
6  	178   	4.91667	1  	5  
7  	184   	4.91333	1  	5  
8  	174   	4.91333	1  	5  
9  	153   	4.96667	1  	5  
10 	184   	4.93   	1  	5  
11 	185   	4.93333	1  	5  
12 	163   	4.92   	1  	5  
13 	167   	4.96667	3  	5  
14 	176   	4.92667	1  	5  
15 	178   	4.91667	1  	5  
16 	161   	4.93667	1  	5  
17 	172   	4.96   	1  	5  
18 	162   	4.93   	1  	5  
19 	180   	4.96333	1  	5  
20 	170   	4.97667	2  	5  
21 	185   	4.95667	1  	5  
22 	185   	4.99   	3  	5  
23 	187   	4.95667	1  	5  
24 	162   	4.97   	1  	5  
25 	170   	4.97333	1  	5  
26 	192   	4.92667	1  	5  
27 	172   	4.93   	1  	5  
28 	152   	4.94667	1  	5  
29 	192   	4.98   	2  	5  
30 	166   	4.99   	2  	5  
31 	169   	4.96667	1  	5  
32 	173   	4.97667	1  	5  
33 	167   	4.98667	2  	5  
34 	181   	4.95   	1  	5  
35 	169   	4.97   	1  	5  
3

array([5.])

In [86]:
text_clf.predict([sentence])


array([5.])